# Number of individuals

In this example, we will be seeing how the number of individuals is influencing the simulation time of tstrait. We will also be comparing the simulation time of tstrait and the simulation framework described in Martin et al. (2017).

In [ ]:
import tstrait
import tskit
import time
import numpy as np
import stdpopsim
import itertools
import pandas as pd

In [ ]:
def compute_time_tstrait(ts):
    times = []
    num_causal = 1000
    trait_model = tstrait.trait_model(distribution="normal", mean=0, var=1)
    for _ in range(10):
        before = time.perf_counter()
        sim_result = tstrait.sim_phenotype(ts, num_causal, trait_model, 0.3)
        duration = time.perf_counter() - before
        times.append(duration)
    return np.array(times)

In [ ]:
species = stdpopsim.get_species("HomSap")
model = stdpopsim.PiecewiseConstantSize(species.population_size)
engine = stdpopsim.get_engine("msprime")

In [ ]:
# tstrait simulation to compile the Python code by using numba
samples = {"pop_0": 100}
contig = species.get_contig(length=1_000_000)
ts = engine.simulate(model, contig, samples)
trait_model = tstrait.trait_model(distribution="normal", mean=0, var=1)
sim_result = tstrait.sim_phenotype(ts, 5, trait_model, 0.3)

## tstrait simulation

In [ ]:
time_result = {}

In [ ]:
# This is separated from the rest, as the sample size is really large
length_array = [50, 100, 200]
num_ind_array = [1, 2, 3, 4, 5]

for length, num_ind in itertools.product(length_array, num_ind_array):
    ts = tskit.load("{}e6_{}Mb_stdpopsim".format(num_ind, length))
    time_result["{}e6_{}Mb".format(num_ind, length)] = compute_time_tstrait(ts)

In [ ]:
time_df = pd.DataFrame(time_result)

In [ ]:
time_df.to_csv("output/tstrait_time.csv")

In [ ]:
length_array = [50, 100, 200]
num_ind_array = [50_000, 100_000, 250_000, 500_000]

for length, num_ind in itertools.product(length_array, num_ind_array):
    ts = tskit.load("{}_{}Mb_stdpopsim".format(num_ind, length))
    time_result["{}_{}Mb".format(num_ind, length)] = compute_time_tstrait(ts)

In [ ]:
time_df = pd.DataFrame(time_result)

In [ ]:
time_df.to_csv("output/tstrait_time_1.csv")

In [ ]:
length_array = [50, 100, 200]
num_ind_array = [1_000, 5_000, 10_000, 25_000]

species = stdpopsim.get_species("HomSap")
model = stdpopsim.PiecewiseConstantSize(species.population_size)
engine = stdpopsim.get_engine("msprime")

for length, num_ind in itertools.product(length_array, num_ind_array):
    samples = {"pop_0": num_ind}
    contig = species.get_contig(length= length * 1_000_000)
    ts = engine.simulate(model, contig, samples)
    time_result["{}_{}Mb".format(num_ind, length)] = compute_time_tstrait(ts)

In [ ]:
time_df = pd.DataFrame(time_result)

In [ ]:
time_df.to_csv("output/tstrait_time_2.csv")

## Martin et al. (2017) simulation

In [ ]:
from genotype_matrix import matrix

In [ ]:
def compute_time_martin(ts):
    times = []
    num_causal = 1000
    for _ in range(10):
        before = time.perf_counter()
        sim_result = matrix(ts, 0.3, 1000, np.random.randint(low=1, high=100_000))
        duration = time.perf_counter() - before
        times.append(duration)
    return np.array(times)

In [ ]:
num_ind_array = [1000, 5000, 10_000, 25_000]

for num_ind in num_ind_array:
    samples = {"pop_0": num_ind}
    contig = species.get_contig(length=100_000_000)
    ts = engine.simulate(model, contig, samples)
    time_result["Martin_{}_{}Mb".format(num_ind, 100)] = compute_time_martin(ts)

In [ ]:
num_ind_array = [50_000, 100_000, 250_000, 500_000]

for num_ind in num_ind_array:
    length = 100
    ts = tskit.load("{}_{}Mb_stdpopsim".format(num_ind, length))
    time_result["Martin_{}_{}Mb".format(num_ind, length)] = compute_time_martin(ts)

In [ ]:
time_df = pd.DataFrame(time_result)

In [ ]:
time_df.to_csv("output/previous_time.csv")